# Ramsey
## f_LO = §f_LO§ GHz
## T2 = §T2§ µs

In [ ]:
path = '006.h5'
f = h5py.File(path, 'r)

print(f.keys())
wait = np.unique(f['data']['wait'][()])*4
print(wait.shape)

freq = np.unique(f['parameters']['freq'])[()]

data1 = f['data']['I1'][()] + 1j * f['data']['Q1'][()]
data2 = f['data']['I2'][()] + 1j * f['data']['Q2'][()]
data = data1-data2
print(data.shape)

data = af.IQ_rot(data)
print(data.shape)

f.close()

In [ ]:
def exp_all_freq(x, t2, f0, b,d):
    t_arr = np.array(x[0])
    f_arr = np.array(x[1])
    t2 = np.abs(t2)
    return b*(np.exp(-t_arr/t2)*np.cos(2*np.pi*t_arr*np.abs(f_arr-f0)+d))


x, y = np.meshgrid(wait, freq)
xdata = np.vstack((x.ravel(), y.ravel()))
ydata = np.real(data).ravel()

popt, pcov = opt.curve_fit(exp_all_freq, xdata, ydata, (13e3, 4.441, 6e-5, 0.))

print(popt)

In [ ]:
fig, ax = plt.subplots(1,1)
wait_fine = np.linspace(wait[0], wait[-1], 1e3)

def up(fr):
    ax.cla()
    ax.plot(wait, np.real(data[fr]), '.-')
    ax.plot(wait_fine, exp_all_freq((wait_fine,freq[fr]), *popt), 'k--')
      
interactive(up, fr = (0, len(freq)-1, 1))